In [147]:
import os
import sys
import json
import copy
from rv_zep import parse_file
import tabulate

OPCODES = {
    'fld': 0,
    'fsd': 1,
    'fadd': 2,
    'fsub': 3,
    'fmul': 4,
    'fdiv': 5
}

UNIT_TYPES = {
    'fld': 'int',
    'fsd': 'int',
    'fadd': 'fadd',
    'fsub': 'fadd',
    'fmul': 'fmul',
    'fdiv': 'fdiv'
}

# Define register prefix constants
REG_PREFIXES = {
    'x': 'int',
    'f': 'float'
}


In [140]:
config_file = open("./unit_config.json")
units_configuration = json.load(config_file)

In [142]:
units_configuration

{'fint': {'number': 1, 'ex_cycles': 1},
 'fmul': {'number': 2, 'ex_cycles': 4},
 'fadd': {'number': 1, 'ex_cycles': 2},
 'fdiv': {'number': 1, 'ex_cycles': 10}}

In [188]:
class ScoreBoard:
    def __init__(self, instructions, configuration):

        self.units_configuration = configuration
        self.instruction_queue = list(enumerate(instructions))
        self.board = self.build_board(self.units_configuration)
        self.ops = {}
        self.clock_counter = 0
        self.solution = self.build_solution(self.instruction_queue)
    
    def tick(self):

        self.issue_instruction()
        self.update_board()
        self.clock_counter += 1

        if self.instruction_queue == [] and not self.board_busy():
            return True
        else:
            return False

    def build_board(self, units_configuration):
        #[name][busy][op][fi][fj][fk][qj][qk][rj][rk][instruction_line][state][state_cycles]
        board = []
        base_unit = {'type':None, 'name': None, 'busy':None, 'op':None, 'fi':None, 'fk':None, 'qj':None, 'qk':None, 'rj':None, 'instruction_line':None,'state':None,'ex_cycles':None}

        for unit in units_configuration:
            unit_number = units_configuration[unit]['number']
            for count in range(unit_number):
                functional_unit = copy.deepcopy(base_unit)
                functional_unit['name'] = unit + str(count)
                functional_unit['type'] = unit
                functional_unit['busy'] = False
                functional_unit['ex_cycles'] = units_configuration[unit]['ex_cycles']
                board.append(functional_unit)
        
        return board

    def build_solution(self, instruction_list):
        instruction = 0
        issue = 1
        read = 2
        ex = 3
        write = 4

        base_solution = ['Name', None, None, None, None]
        header = copy.deepcopy(base_solution)
        header[issue] = 'Issue'
        header[read] = 'Read'
        header[ex] = 'Ex'
        header[write] = 'Write'
        solution = []
        solution.append(header)

        for current_instruction in instruction_list:
            instruction_solution = copy.deepcopy(base_solution)
            instruction_solution[instruction] = current_instruction[1]['opcode']
            solution.append(instruction_solution)
        return solution

    
    def update_board(self):
        #[name][busy][op][fi][fj][fk][qj][qk][rj][rk][instruction_line][state][state_cycles]

        """ for unit in units_in_board:
            
            can_change_state = check dependencies and ex times
            if can_change_state:
                change_state(self.unit_configuration[unit])
                write_to_solution(self.clock_counter)
            if finished_writeback:
                clean_unit() """
        ...
    
    def issue_instruction(self):
        instruction_peek = self.instruction_queue[0][1]
        unit_type = UNIT_TYPES[self.opcode_to_instruction(instruction_peek['opcode'])]
        instruction_destination = instruction_peek['rd']
        for unit in self.board:
            print(f'{unit['type']} and { unit_type}')
            if unit['type'] == unit_type:
                if unit['busy'] == False and self.hazard_check(instruction_destination) != False:
                    element = self.instruction_queue.pop(0)
                    instruction = element[1]
                    instruction_line = element[0]
                    self.set_instruction(instruction, unit)
                    self.update_solution(instruction_line,'issue')

    def update_solution(self, line, type):
        line = line + 1
        self.solution[line][type] = self.clock_counter

    def opcode_to_instruction(self,opcode):
        instruction_name = list(OPCODES.keys())[opcode]
        return instruction_name

    
    def set_instruction(self, instruction, unit):
        ...

    
    def hazard_check(self, fi):
        hazard = False
        for unit in self.board:
            if unit['fi'] == fi:
                hazard = True
        return hazard

    def board_busy(self):
        busy = False
        for unit in self.board:
            busy = unit['busy'] or busy
        return busy

    def print_board(self):
        print(tabulate.tabulate(self.board))

    def print_solution(self):
        print(tabulate.tabulate(self.solution))

    def break_board(self):
        for unit in self.board:
            if unit['type'] == 'fmult':
                if unit[ 'name'] == 'fmult1':
                    unit['instruction_line'] = 13
                    print(unit)

    def run(self):
        
        self.done = False
        while not self.done:
            self.tick()
        self.print_solution()
        

In [189]:
config_file = open("./unit_config.json")
configuration = json.load(config_file)
instructions = parse_file('example.s')

board = ScoreBoard(instructions, configuration)
board.print_board()

----  -----  -----                  --
fint  fint0  False                   1
fmul  fmul0  False                   4
fmul  fmul1  False                   4
fadd  fadd0  False                   2
fdiv  fdiv0  False                  10
----  -----  -----                  --


In [190]:
board.issue_instruction()

fint and fint
fmul and fint
fmul and fint
fadd and fint
fdiv and fint


In [164]:
board.print_board()

----  -----  -----                  --
fint  fint0  False                   1
fmul  fmul0  False                   4
fmul  fmul1  False                   4
fadd  fadd0  False                   2
fdiv  fdiv0  False                  10
----  -----  -----                  --


In [163]:
board.print_solution()

----  -----  ----  --  -----
Name  Issue  Read  Ex  Write
0
0
5
4
2
1
----  -----  ----  --  -----


In [82]:
board.board

[{'type': 'fint',
  'name': 'fint0',
  'busy': False,
  'op': None,
  'fi': None,
  'fk': None,
  'qj': None,
  'qk': None,
  'rj': None,
  'instruction_line': None,
  'state': None,
  'ex_cycles': 1},
 {'type': 'fmult',
  'name': 'fmult0',
  'busy': False,
  'op': None,
  'fi': None,
  'fk': None,
  'qj': None,
  'qk': None,
  'rj': None,
  'instruction_line': None,
  'state': None,
  'ex_cycles': 4},
 {'type': 'fmult',
  'name': 'fmult1',
  'busy': False,
  'op': None,
  'fi': None,
  'fk': None,
  'qj': None,
  'qk': None,
  'rj': None,
  'instruction_line': 13,
  'state': None,
  'ex_cycles': 4},
 {'type': 'fadd',
  'name': 'fadd0',
  'busy': False,
  'op': None,
  'fi': None,
  'fk': None,
  'qj': None,
  'qk': None,
  'rj': None,
  'instruction_line': None,
  'state': None,
  'ex_cycles': 2},
 {'type': 'fdiv',
  'name': 'fdiv0',
  'busy': False,
  'op': None,
  'fi': None,
  'fk': None,
  'qj': None,
  'qk': None,
  'rj': None,
  'instruction_line': None,
  'state': None,
  'ex_c

In [77]:
for unit in board.board:
    if unit['type'] == 'fmult':
        if unit[ 'name'] == 'fmult1':
            unit['instruction_line'] = 13
            print(unit)
     
    

{'type': 'fmult', 'name': 'fmult1', 'busy': False, 'op': None, 'fi': None, 'fk': None, 'qj': None, 'qk': None, 'rj': None, 'instruction_line': 13, 'state': None, 'ex_cycles': 4}


In [46]:
for unit in units_configuration:
    print(units_configuration[unit]['ex_cycles'])

1
4
2
10


In [117]:
OPCODES.keys()[OPCODES.values().index(0)]

AttributeError: 'dict_values' object has no attribute 'index'

In [138]:
list(OPCODES.keys())

['fld', 'fsd', 'fadd', 'fsub', 'fmul', 'fdiv']

In [103]:
enum[0][1]

{'opcode': 0,
 'rs1': 1,
 'rs1_type': 'int',
 'rs2': 0,
 'rs2_type': None,
 'rd': 1,
 'rd_type': 'float',
 'imm': 0}

In [29]:
instructions

[{'opcode': 0,
  'rs1': 1,
  'rs1_type': 'int',
  'rs2': 0,
  'rs2_type': None,
  'rd': 1,
  'rd_type': 'float',
  'imm': 0},
 {'opcode': 0,
  'rs1': 1,
  'rs1_type': 'int',
  'rs2': 0,
  'rs2_type': None,
  'rd': 5,
  'rd_type': 'float',
  'imm': 0},
 {'opcode': 5,
  'rs1': 4,
  'rs1_type': 'float',
  'rs2': 5,
  'rs2_type': 'float',
  'rd': 2,
  'rd_type': 'float',
  'imm': None},
 {'opcode': 4,
  'rs1': 8,
  'rs1_type': 'float',
  'rs2': 9,
  'rs2_type': 'float',
  'rd': 4,
  'rd_type': 'float',
  'imm': None},
 {'opcode': 2,
  'rs1': 2,
  'rs1_type': 'float',
  'rs2': 3,
  'rs2_type': 'float',
  'rd': 1,
  'rd_type': 'float',
  'imm': None},
 {'opcode': 1,
  'rs1': 2,
  'rs1_type': 'int',
  'rs2': 4,
  'rs2_type': 'float',
  'rd': 0,
  'rd_type': None,
  'imm': 0}]

In [14]:
a = {'banana':1,'orange':5}

In [15]:
a.get(5)